# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [3]:
estimation_method = 'hardware-capex-opex'
compute_percentile_threshold = 75  # select systems >= this percentile of training compute
variant = 'linearly-adjust-to-release-price-and-tpu-versions-distinguished'

estimation_method_lookup = {
    'cloud': estimate_costs,
    'amortized': estimate_amortized_hardware_costs,
    'up-front-server-capex': estimate_upfront_server_capex,
    'hardware-capex-opex': estimate_hardware_capex_opex,
}
cost_estimation_function = estimation_method_lookup[estimation_method]

results_dir = f'results/{estimation_method}-{compute_percentile_threshold}th-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_percentile_threshold=compute_percentile_threshold
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(76, 5510, 142)

# Cost estimation

In [6]:
cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

==== System: Gemini Ultra ====
Found server price for Google TPU v4 at 2023-06-29 00:00:00: 19390.4

==== System: Qwen-72B ====
Could not find hardware model for Qwen-72B


==== System: Inflection-2 ====
No training time found, assuming 32.083333333333336

Found server price for NVIDIA H100 SXM5 at 2023-05-29 00:00:00: 55800.16974726519

==== System: Yi-34B ====
Could not find hardware model for Yi-34B


==== System: ChatGLM3 ====
Could not find hardware model for ChatGLM3


==== System: Falcon 180B ====
Found server price for NVIDIA A100 SXM4 40 GB at 2022-01-01 00:00:00: 43939.84962406015

==== System: Llama 2-70B ====
Found server price for NVIDIA A100 SXM4 80 GB at 2022-01-20 00:00:00: 34357.91400675929

==== System: Llama 2-34B ====
No training time found, assuming 32.083333333333336

Found server price for NVIDIA A100 SXM4 80 GB at 2022-01-20 00:00:00: 34357.91400675929

==== System: Claude 2 ====
Could not find hardware model for Claude 2


==== System: xTrimoPGLM -100B ====
Fou

In [7]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Batch size notes,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Cost
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,252.0,Gemini: A Family of Highly Capable Multimodal ...,...,NaN,Industry,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN,1.148555e+08
46,Qwen-72B,Language,"Chat,Code generation","Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Ka...",SOTA improvement,"SOTA on several Chinese benchmarks, with highe...",Permissive license (depr.),https://huggingface.co/Qwen/Qwen-72B,NaN,NaN,...,Table 1 https://arxiv.org/abs/2309.16609\n(thi...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,API access,https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,NaN,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN,1.901656e+07
83,Yi-34B,Language,Chat,NaN,Significant use,2nd most popular model on HuggingFace: https:/...,Permissive license (depr.),https://arxiv.org/abs/2403.04652,NaN,NaN,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,ChatGLM3,Multimodal,"Chat,Visual question answering",NaN,SOTA improvement,"Aiming at GPT-4V, ChatGLM3 has implemented ite...",NaN,https://www.zhipuai.cn/en/news/76,NaN,Zhipu AI launches third-generation base model,...,NaN,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,Xception,Vision,Image classification,François Chollet,Highly cited,NaN,NaN,https://arxiv.org/abs/1610.02357,11338.0,Xception: Deep Learning with Depthwise Separab...,...,NaN,Industry,NaN,NaN,NaN,43200.0,NaN,NaN,NaN,1.917107e+04
1095,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,6105.0,Google's Neural Machine Translation System: Br...,...,NaN,Industry,NaN,NaN,NaN,414720.0,NaN,NaN,NaN,1.840423e+05
1131,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14733.0,Mastering the game of Go with deep neural netw...,...,NaN,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2741.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,Industry,NaN,NaN,NaN,301.0,NaN,NaN,NaN,2.376194e+02


In [8]:
cost_df['Cost'].notna().sum()

55

Use the below to check data availability for specific systems

In [9]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [10]:
# TODO move to function

In [11]:
cost_df['Publication date']

39     2023-12-06
46     2023-11-30
54     2023-11-22
83     2023-11-02
92     2023-10-27
          ...    
1094   2016-10-07
1095   2016-09-26
1131   2016-01-27
1137   2015-12-08
1141   2015-10-01
Name: Publication date, Length: 76, dtype: datetime64[ns]

In [12]:
from_year_month = cost_df['Publication date'].apply(str)
cost_df['Publication date'] = from_year_month

In [13]:
cost_df['Publication date']

39      2023-12-06 00:00:00
46      2023-11-30 00:00:00
54      2023-11-22 00:00:00
83      2023-11-02 00:00:00
92      2023-10-27 00:00:00
               ...         
1094    2016-10-07 00:00:00
1095    2016-09-26 00:00:00
1131    2016-01-27 00:00:00
1137    2015-12-08 00:00:00
1141    2015-10-01 00:00:00
Name: Publication date, Length: 76, dtype: object

In [14]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [15]:
cost_df['Cost (inflation-adjusted)'].dropna()

39      1.148555e+08
54      1.906107e+07
135     3.530364e+07
174     3.381179e+06
175     1.516304e+06
186     5.991327e+06
224     1.738009e+07
263     1.142364e+08
279     2.395916e+06
285     9.494968e+05
323     1.239271e+07
329     1.222735e+06
334     1.722338e+06
349     7.521468e+06
374     3.350201e+06
375     2.238868e+06
396     8.089851e+06
402     9.471124e+05
436     3.369732e+05
439     1.286161e+06
440     4.895098e+05
448     7.440549e+06
470     1.140153e+06
472     2.482744e+05
499     1.252483e+06
503     3.010045e+06
542     7.523424e+06
549     7.145105e+05
569     3.751311e+05
591     3.904798e+05
599     3.552884e+05
637     2.162117e+05
652     5.330149e+05
655     1.987343e+05
715     8.317760e+05
717     2.303021e+05
724     4.208945e+06
759     1.097811e+05
764     5.983208e+05
765     2.639104e+04
791     3.339794e+05
795     2.032130e+05
809     3.639313e+05
810     2.017062e+05
835     1.488732e+05
918     8.696712e+03
921     1.023505e+04
982     5.773

In [16]:
cost_df['Cost (inflation-adjusted)'].notna().sum()

55

# Regression

In [17]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [18]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.540
Model:                            OLS   Adj. R-squared:                  0.531
Method:                 Least Squares   F-statistic:                     62.17
Date:                Mon, 06 May 2024   Prob (F-statistic):           1.71e-10
Time:                        12:08:13   Log-Likelihood:                -57.271
No. Observations:                  55   AIC:                             118.5
Df Residuals:                      53   BIC:                             122.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -714.7303     91.392     -7.820      0.000    -898.040    -531.421
x1             0.3565      0.045      7.885      0.000       0.266       0.447
==============================================================================
Omnibus:                        2.171   Durbin-Watson:                   1.715
Prob(Omnibus):                  0.338   Jarque-Bera (JB):                1.604
Skew:                           0.415   Prob(JB):                        0.448
Kurtosis:                       3.111   Cond. No.                     1.96e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.96e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
print_growth_rates(reg_results)

0.36 OOMs/year (95% CI: 0.27, 0.45)
2.3x/year (95% CI: 1.8x, 2.8x)
doubling time of 10 months (95% CI: 8, 14)


In [20]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [21]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,3.712391,0.291358,3.128001,4.296780,2.194780,5.230001,2015.00000,2015-01-01
1,3.748406,0.287039,3.172678,4.324133,2.234109,5.262702,2015.10101,2015-02-06
2,3.784420,0.282728,3.217339,4.351501,2.273390,5.295450,2015.20202,2015-03-15
3,3.820435,0.278426,3.261984,4.378886,2.312623,5.328248,2015.30303,2015-04-21
4,3.856450,0.274131,3.306612,4.406288,2.351807,5.361094,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,7.133804,0.184097,6.764552,7.503055,5.685364,8.582243,2024.59596,2024-08-06
96,7.169819,0.188036,6.792666,7.546971,5.719345,8.620293,2024.69697,2024-09-12
97,7.205833,0.192004,6.820723,7.590944,5.753270,8.658397,2024.79798,2024-10-19
98,7.241848,0.195997,6.848728,7.634968,5.787141,8.696556,2024.89899,2024-11-25


# Plots

In [22]:
title_suffix = f' to train top {100 - compute_percentile_threshold}% most compute-intensive ML systems'
plot_title_lookup = {
    'cloud': 'Cloud compute cost' + title_suffix,
    'amortized': 'Amortized cost of hardware' + title_suffix,
    'up-front-server-capex': 'Acquisition cost of hardware' + title_suffix,
    'hardware-capex-opex': 'Hardware CapEx + OpEx' + title_suffix,
}

In [23]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text=plot_title_lookup[estimation_method])

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [24]:
cost_df['Training hardware'].str.contains('TPU', na=False)

39       True
46      False
54      False
83      False
92      False
        ...  
1094    False
1095    False
1131    False
1137    False
1141    False
Name: Training hardware, Length: 76, dtype: bool

In [25]:
label_systems = ['GNMT', 'DALL-E', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra']

tpu_mask = cost_df['Training hardware'].str.contains('TPU', na=False)
tpu_cost_df = cost_df.loc[tpu_mask]
gpu_cost_df = cost_df.loc[~tpu_mask]

fig = px.scatter(
    gpu_cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    log_y=True,
)
fig.add_scatter(
    x=tpu_cost_df['Publication date'],
    y=tpu_cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    marker_symbol='circle-open',
    name='Using equivalent price of TPU',
)
fig.add_scatter(
    x=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=gpu_cost_df.loc[gpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)
fig.add_scatter(
    x=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Publication date'],
    y=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=tpu_cost_df.loc[tpu_cost_df['System'].isin(label_systems)]['System'],
    mode='text',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])
fig.update_yaxes(range=[1, 9])

# legend on top
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='top',
    y=-0.15,
    xanchor='center',
    x=0.5,
))

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
fig.update_layout(title_text=plot_title_lookup[estimation_method])

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Export data

In [26]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06 00:00:00,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,1.148555e+08,1.148555e+08
46,Qwen-72B,Language,"Chat,Code generation",Permissive license (depr.),NaN,2023-11-30 00:00:00,Alibaba,7.200000e+10,1.300000e+24,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,API access,Inflection-2: The Next Step Up,2023-11-22 00:00:00,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,1.901656e+07,1.906107e+07
83,Yi-34B,Language,Chat,Permissive license (depr.),NaN,2023-11-02 00:00:00,01.AI,3.400000e+10,6.100000e+23,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,ChatGLM3,Multimodal,"Chat,Visual question answering",NaN,Zhipu AI launches third-generation base model,2023-10-27 00:00:00,Zhipu AI,1.300000e+11,1.092000e+24,1.050000e+12,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,Xception,Vision,Image classification,NaN,Xception: Deep Learning with Depthwise Separab...,2016-10-07 00:00:00,Google,2.285595e+07,4.360000e+20,3.500000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,60.0,NaN,NaN,NaN,1.917107e+04,2.093516e+04
1095,GNMT,Language,Translation,NaN,Google's Neural Machine Translation System: Br...,2016-09-26 00:00:00,Google,2.780000e+08,6.900000e+21,3.600000e+08,...,NVIDIA Tesla K80,United States of America,NaN,NaN,96.0,NaN,NaN,NaN,1.840423e+05,2.009776e+05
1131,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27 00:00:00,DeepMind,NaN,1.900000e+21,2.940000e+07,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,NaN,Deep Speech 2: End-to-End Speech Recognition i...,2015-12-08 00:00:00,Baidu Research - Silicon Valley AI Lab,3.800000e+07,2.600000e+19,1.633392e+08,...,NVIDIA GTX Titan X,United States of America,NaN,NaN,NaN,0.45,NaN,NaN,2.376194e+02,2.643349e+02


In [27]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)